# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, LinearColorMapper, WMTSTileSource, FixedTicker
from bokeh.transform import linear_cmap
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook
from bokeh.palettes import Viridis256

# Import API key
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name,size,x,y
14,port elizabeth,ZA,-33.9180,25.5701,58,Waterford Hotel,10,2.846451e+06,-4.017797e+06
20,koungou,YT,-12.7336,45.2042,69,Hôtel Trévani,15,5.032109e+06,-1.429313e+06
31,adamstown,PN,-25.0660,-130.1015,57,No hotel found,10,-1.448283e+07,-2.883853e+06
33,vilyuchinsk,RU,52.9306,158.4028,63,Победа,15,1.763332e+07,6.970171e+06
35,moreira sales,BR,-24.0622,-53.0069,49,No hotel found,10,-5.900701e+06,-2.760989e+06


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Define size bins for humidity
humidity_bins = [0, 40, 60, 80, 100]
humidity_labels = [5, 10, 15, 20]
city_data_df['size'] = pd.cut(city_data_df['Humidity'], bins=humidity_bins, labels=humidity_labels, include_lowest=True)


city_data_df['x'], city_data_df['y'] = zip(*city_data_df.apply(lambda row: wgs84_to_web_mercator(row['Lat'], row['Lng']), axis=1))

# Prepare the data
source = ColumnDataSource(data=dict(
    lat=city_data_df['Lat'],
    lon=city_data_df['Lng'],
    x=city_data_df['x'],
    y=city_data_df['y'],
    humidity=city_data_df['Humidity'],
    city=city_data_df['City'],
    country=city_data_df['Country'],
    size=city_data_df['size']
))


# Define the map plot
output_notebook()

# Custom tile source for OpenStreetMap.CH
tile_source = WMTSTileSource(url='https://tile.openstreetmap.org/{z}/{x}/{y}.png')

# Create the figure
p = figure(title="City Humidity Map", x_axis_type="mercator", y_axis_type="mercator", 
           width=800, height=600)
p.add_tile(tile_source)

# Define color mapper using linear_cmap
color_mapper = LinearColorMapper(palette=Viridis256, low=min(city_data_df['Humidity']), high=max(city_data_df['Humidity']))
color_mapping = linear_cmap(field_name='humidity', palette=Viridis256, low=min(city_data_df['Humidity']), high=max(city_data_df['Humidity']))

# Add circle for each city, size is related to humidity and color is related to humidity
p.circle(x='x', y='y', size='size', color=color_mapping, fill_alpha=0.6, source=source)

# Add hover tool
hover = HoverTool()
hover.tooltips = [("City", "@city"), ("Country", "@country"), ("Humidity", "@humidity%")]
p.add_tools(hover)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, width=8, height=500, location=(0,0), title='Humidity')
p.add_layout(color_bar, 'right')

# Set axis labels to latitude and longitude
p.xaxis.axis_label = "Longitude"
p.yaxis.axis_label = "Latitude"

# Set major ticks for latitude and longitude in degrees
x_ticks = [-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150]
y_ticks = [-80, -60, -30, 0, 30, 60, 80]

p.xaxis.ticker = FixedTicker(ticks=[wgs84_to_web_mercator(0, lon)[0] for lon in x_ticks])
p.yaxis.ticker = FixedTicker(ticks=[wgs84_to_web_mercator(lat, 0)[1] for lat in y_ticks])

# Set axis formatting
p.xaxis.major_label_overrides = {wgs84_to_web_mercator(0, lon)[0]: str(lon) for lon in x_ticks}
p.yaxis.major_label_overrides = {wgs84_to_web_mercator(lat, 0)[1]: str(lat) for lat in y_ticks}

# Show the plot
show(p)


Loading BokehJS ...

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities_df = city_data_df[(city_data_df['Max Temp'] <= 85) & 
                                  (city_data_df['Max Temp'] >= 50) & 
                                  (city_data_df['Humidity'] <= 70)]

# Drop any rows with null values
my_cities_df = my_cities_df.dropna()

# Display sample data
my_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,size,x,y
14,14,port elizabeth,-33.9180,25.5701,66.31,58,0,3.44,ZA,24-06-29,10,2.846451e+06,-4.017797e+06
20,20,koungou,-12.7336,45.2042,76.71,69,20,13.80,YT,24-06-29,15,5.032109e+06,-1.429313e+06
31,31,adamstown,-25.0660,-130.1015,67.57,57,50,28.45,PN,24-06-29,10,-1.448283e+07,-2.883853e+06
33,33,vilyuchinsk,52.9306,158.4028,68.27,63,100,6.35,RU,24-06-29,15,1.763332e+07,6.970171e+06
35,35,moreira sales,-24.0622,-53.0069,51.85,49,89,11.41,BR,24-06-29,10,-5.900701e+06,-2.760989e+06


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,port elizabeth,ZA,-33.9180,25.5701,58,
20,koungou,YT,-12.7336,45.2042,69,
31,adamstown,PN,-25.0660,-130.1015,57,
33,vilyuchinsk,RU,52.9306,158.4028,63,
35,moreira sales,BR,-24.0622,-53.0069,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('Geoapify_API')
token = api_key

# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    params = {
    "categories": "accommodation.hotel",
    "filter": f"circle:{lon},{lat},{radius}",
    "bias": f"proximity:{lon},{lat}",
    "limit": 1,
    "apiKey": api_key  
    
}
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
koungou - nearest hotel: Hôtel Trévani
adamstown - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: Победа
moreira sales - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
sorsk - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
deniliquin - nearest hotel: No hotel found
inongo - nearest hotel: Chez LELE IYOLO
the pas - nearest hotel: Wescana Inn
ust-nera - nearest hotel: гостиница "Солнечная"
elesbao veloso - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
aykhal - nearest hotel: Уют
durban - nearest hotel: Tudor
xingcheng - nearest hotel: No hotel found
yulinshi - nearest hotel: 榆溪大酒店
papatowai - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
bilibino - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
antsiranana - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
14,port elizabeth,ZA,-33.9180,25.5701,58,Waterford Hotel
20,koungou,YT,-12.7336,45.2042,69,Hôtel Trévani
31,adamstown,PN,-25.0660,-130.1015,57,No hotel found
33,vilyuchinsk,RU,52.9306,158.4028,63,Победа
35,moreira sales,BR,-24.0622,-53.0069,49,No hotel found
...,...,...,...,...,...,...
534,ak-suu,KG,42.4995,78.5270,51,BellavistaHotel
538,kosh-agach,RU,50.0000,88.6667,45,Заря
544,valparaiso,CL,-33.0393,-71.6273,63,Ibis Hotel
549,samalut,EG,28.3121,30.7101,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Define size bins for humidity
humidity_bins = [0, 40, 60, 80, 100]
humidity_labels = [5, 10, 15, 20]
hotel_df['size'] = pd.cut(hotel_df['Humidity'], bins=humidity_bins, labels=humidity_labels, include_lowest=True)


hotel_df['x'], hotel_df['y'] = zip(*hotel_df.apply(lambda row: wgs84_to_web_mercator(row['Lat'], row['Lng']), axis=1))

# Prepare the data
source = ColumnDataSource(data=dict(
    lat=hotel_df['Lat'],
    lon=hotel_df['Lng'],
    x=hotel_df['x'],
    y=hotel_df['y'],
    humidity=hotel_df['Humidity'],
    hotel=hotel_df['Hotel Name'],
    city=hotel_df['City'],
    country=hotel_df['Country'],
    size=hotel_df['size']
))


# Define the map plot
output_notebook()

# Custom tile source for OpenStreetMap.CH
tile_source = WMTSTileSource(url='https://tile.openstreetmap.org/{z}/{x}/{y}.png')

# Create the figure
p = figure(title="Potential City Vacation Map", x_axis_type="mercator", y_axis_type="mercator", 
           width=800, height=600)
p.add_tile(tile_source)

# Define color mapper using linear_cmap
color_mapper = LinearColorMapper(palette=Viridis256, low=min(hotel_df['Humidity']), high=max(hotel_df['Humidity']))
color_mapping = linear_cmap(field_name='humidity', palette=Viridis256, low=min(hotel_df['Humidity']), high=max(hotel_df['Humidity']))

# Add circle for each city, size is related to humidity and color is related to humidity
p.circle(x='x', y='y', size='size', color=color_mapping, fill_alpha=0.6, source=source)

# Add hover tool
hover = HoverTool()
hover.tooltips = [("City", "@city"), ("Country", "@country"), ("Humidity", "@humidity%"), ('Hotel Name', '@hotel')]
p.add_tools(hover)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, width=8, height=500, location=(0,0), title='Humidity')
p.add_layout(color_bar, 'right')

# Set axis labels to latitude and longitude
p.xaxis.axis_label = "Longitude"
p.yaxis.axis_label = "Latitude"

# Set major ticks for latitude and longitude in degrees
x_ticks = [-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150]
y_ticks = [-80, -60, -30, 0, 30, 60, 80]

p.xaxis.ticker = FixedTicker(ticks=[wgs84_to_web_mercator(0, lon)[0] for lon in x_ticks])
p.yaxis.ticker = FixedTicker(ticks=[wgs84_to_web_mercator(lat, 0)[1] for lat in y_ticks])

# Set axis formatting
p.xaxis.major_label_overrides = {wgs84_to_web_mercator(0, lon)[0]: str(lon) for lon in x_ticks}
p.yaxis.major_label_overrides = {wgs84_to_web_mercator(lat, 0)[1]: str(lat) for lat in y_ticks}

# Show the plot
show(p)


Loading BokehJS ...